# Retriever Test

In [1]:
import json
import os
import pickle
import time
from contextlib import contextmanager
from typing import List, NoReturn, Optional, Tuple, Union

import faiss
import numpy as np
import pandas as pd
from datasets import Dataset, concatenate_datasets, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.auto import tqdm

import argparse
import utils
from pprint import pprint
import konlpy.tag
from transformers import AutoTokenizer
from importlib import import_module
import json

## 1. Loading parameters

In [2]:
MYDICT = {'key': 'value'}

parser = argparse.ArgumentParser(description="")
parser.add_argument(
    "--retriever_path",
    default="",
    metavar="", type=str, help=""
)
parser.add_argument(
    "--config_retrieval",
    default="./config/retrieval_config.json",
    metavar="./config/retrieval_config.json", type=str, help=""
)
parser.add_argument(
    "--vectorizer_type",
    default="TfidfVectorizer",
    metavar="TfidfVectorizer", type=str, help=""
)
parser.add_argument('--vectorizer_parameters',
                    type=json.loads, default=MYDICT)
parser.add_argument(
    "--tokenizer_type",
    default="AutoTokenizer",
    metavar="AutoTokenizer", type=str, help=""
)
parser.add_argument(
    "--dataset_name", default = "./data/train_dataset",
    metavar="./data/train_dataset", type=str, help=""
)
parser.add_argument(
    "--model_name_or_path",
    default ="bert-base-multilingual-cased",
    metavar="bert-base-multilingual-cased",
    type=str,
    help="",
)
parser.add_argument(
    "--top_k",
    default =10,
    metavar=10,
    type=int,
    help="",
)
parser.add_argument("--data_path",default = "./data",
                    metavar="./data", type=str, help="")
parser.add_argument(
    "--context_path",
    default = "wikipedia_documents.json",
    metavar="wikipedia_documents.json", type=str, help=""
)
parser.add_argument(
    "--output_path",
    default="./retriever_result",
    metavar="./retriever_result", type=str, help=""
)

parser.add_argument("--use_faiss", default=False, metavar=False, type=bool, help="")
parser.add_argument("--num_clusters", default=64, metavar=64, type=int, help="")

_StoreAction(option_strings=['--num_clusters'], dest='num_clusters', nargs=None, const=None, default=64, type=<class 'int'>, choices=None, help='', metavar=64)

In [3]:
args = parser.parse_args([])
config = utils.read_json(args.config_retrieval)
parser.set_defaults(**config)
args = parser.parse_args([])

In [4]:
pprint(vars(args))

{'config_retrieval': './config/retrieval_config.json',
 'context_path': 'wikipedia_documents.json',
 'data_path': '../data',
 'dataset_name': '../data/train_dataset',
 'model_name_or_path': 'bert-base-multilingual-cased',
 'num_clusters': 64,
 'output_path': './retriever_result',
 'retriever_path': '',
 'tokenizer_type': 'AutoTokenizer',
 'top_k': 10,
 'use_faiss': False,
 'vectorizer_parameters': {'ngram_range': [1, 2]},
 'vectorizer_type': 'TfidfVectorizer'}


## 2. loading dataset

In [5]:
org_dataset = load_from_disk(args.dataset_name)
full_ds = concatenate_datasets(
    [
        org_dataset["train"].flatten_indices(),
        org_dataset["validation"].flatten_indices(),
    ]
)  # train dev 를 합친 4192 개 질문에 대해 모두 테스트
print("*" * 40, "query dataset", "*" * 40)
print(full_ds)

Loading cached processed dataset at ../data/train_dataset/train/cache-9681ee696ea809ac.arrow
Loading cached processed dataset at ../data/train_dataset/validation/cache-39f91efb8d01b7c9.arrow


**************************************** query dataset ****************************************
Dataset({
    features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
    num_rows: 4192
})


In [6]:
print(len(full_ds["question"]))
print(len(full_ds["context"]))
print(len(full_ds["answers"]))

4192
4192
4192


In [7]:
for index, (q, c, a) in enumerate(zip(full_ds["question"], full_ds["context"], full_ds["answers"])):
    print(f'question : {q}')
    print(f'contenxt : {c}')
    print(f'answers : {a}')
    if index == 5:
        break

question : 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?
contenxt : 미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국 의회의 상원이다.\n\n미국 부통령이 상원의장이 된다. 각 주당 2명의 상원의원이 선출되어 100명의 상원의원으로 구성되어 있다. 임기는 6년이며, 2년마다 50개주 중 1/3씩 상원의원을 새로 선출하여 연방에 보낸다.\n\n미국 상원은 미국 하원과는 다르게 미국 대통령을 수반으로 하는 미국 연방 행정부에 각종 동의를 하는 기관이다. 하원이 세금과 경제에 대한 권한, 대통령을 포함한 대다수의 공무원을 파면할 권한을 갖고 있는 국민을 대표하는 기관인 반면 상원은 미국의 주를 대표한다. 즉 캘리포니아주, 일리노이주 같이 주 정부와 주 의회를 대표하는 기관이다. 그로 인하여 군대의 파병, 관료의 임명에 대한 동의, 외국 조약에 대한 승인 등 신속을 요하는 권한은 모두 상원에게만 있다. 그리고 하원에 대한 견제 역할(하원의 법안을 거부할 권한 등)을 담당한다. 2년의 임기로 인하여 급진적일 수밖에 없는 하원은 지나치게 급진적인 법안을 만들기 쉽다. 대표적인 예로 건강보험 개혁 당시 하원이 미국 연방 행정부에게 퍼블릭 옵션(공공건강보험기관)의 조항이 있는 반면 상원의 경우 하원안이 지나치게 세금이 많이 든다는 이유로 퍼블릭 옵션 조항을 제외하고 비영리건강보험기관이나 보험회사가 담당하도록 한 것이다. 이 경우처럼 상원은 하원이나 내각책임제가 빠지기 쉬운 국가들의 국회처럼 걸핏하면 발생하는 의회의 비정상적인 사태를 방지하는 기관이다. 상원은 급박한 처리사항의 경우가 아니면 법안을 먼저 내는 경우가 드물고 하원이 만든 법안을 수정하여 다시 하원에 되돌려보낸다. 이러한 방식으로 단원제가 빠지기 쉬운 함정을 미리 방지하는 것이다.날짜=2017-02-05
answers : {'answer_start': [235], 'text': ['하원']}
question : 현대적 인사조직관리의 시발점이 된 책은

## 3. Loading Tokenizer

In [8]:
if hasattr(import_module("transformers"), args.tokenizer_type):
    tokenizer_type = getattr(import_module("transformers"), args.tokenizer_type)
    tokenizer = tokenizer_type.from_pretrained(args.model_name_or_path, use_fast=False, )
    print(f'{args.tokenizer_type}')
elif hasattr(import_module("konlpy.tag"), args.tokenizer_type):
    tokenizer = getattr(import_module("konlpy.tag"), args.tokenizer_type)()
    print(f'{args.tokenizer_type}')
else:
    raise Exception(f"Use correct tokenizer type - {args.tokenizer_type}")

AutoTokenizer


In [9]:
print(tokenizer)

PreTrainedTokenizer(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


## 4. Setting Output Directory

In [10]:
if args.tokenizer_type == "AutoTokenizer":
    output_path = args.output_path + f'/{args.vectorizer_type}_{args.model_name_or_path}_{args.context_path}'
else:
    output_path = args.output_path + f'/{args.vectorizer_type}_{args.tokenizer_type}_{args.context_path}'
output_path = utils.increment_directory(output_path)
print(f'output_path directory: {output_path}')

output_path directory: ./retriever_result/TfidfVectorizer_bert-base-multilingual-cased_wikipedia_documents.json_5/


## 5. Initializing SparseRetrieval

In [11]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f"[{name}] done in {time.time() - t0:.3f} s")

class SparseRetrieval:
    def __init__(
        self,
        retrieval_path,
        retrieval_type,
        retrieval_parameters,
        tokenize_fn,
        output_path,
        data_path: Optional[str] = "../data/",
        context_path: Optional[str] = "wikipedia_documents.json",
        num_clusters = 64
    ) -> NoReturn:

        """
        Arguments:
            tokenize_fn:
                기본 text를 tokenize해주는 함수입니다.
                아래와 같은 함수들을 사용할 수 있습니다.
                - lambda x: x.split(' ')
                - Huggingface Tokenizer
                - konlpy.tag의 Mecab

            data_path:
                데이터가 보관되어 있는 경로입니다.

            context_path:
                Passage들이 묶여있는 파일명입니다.

            data_path/context_path가 존재해야합니다.

        Summary:
            Passage 파일을 불러오고 TfidfVectorizer를 선언하는 기능을 합니다.
        """

        self.data_path = data_path
        with open(os.path.join(data_path, context_path), "r", encoding="utf-8") as f:
            wiki = json.load(f)

        # 순서대로 중복 제거
        # ('key1', 'key2', 'key3', 'key1', 'key4', 'key2') -> ['key1', 'key2', 'key3', 'key4']
        self.contexts = list(
            dict.fromkeys([v["text"] for v in wiki.values()])
        )  # set 은 매번 순서가 바뀌므로

        print(f"Lengths of unique contexts : {len(self.contexts)}")
        self.ids = list(range(len(self.contexts)))

        self.p_embedding = None  # get_sparse_embedding()로 생성합니다
        self.indexer = None  # build_faiss()로 생성합니다.
        self.num_clusters = num_clusters

        self.get_sparse_embedding(retrieval_path, retrieval_type, tokenize_fn, retrieval_parameters, output_path)

    def get_sparse_embedding(self, retriever_path, vectorizer_type, tokenize_fn, vectorizer_parameters, output_path) -> NoReturn:

        """
        Summary:
            retriever_path 존재하면 해당 sparse retrieval loading
            retriever_path 존재하지 않으면,
                1) self.vectorizer 호출
                2) Passage Embedding을 만들고
                3) TFIDF와 Embedding을 pickle로 저장합니다.
        """

        # Pickle을 저장합니다.
        pickle_name = f"sparse_embedding.bin"
        vectorizer_name = f"vectorizer.bin"

        if retriever_path:
            print(f'Initializing sparse retriever on {retriever_path}')
            emb_path = os.path.join(retriever_path, pickle_name)
            vectorizer_path = os.path.join(self.data_path, vectorizer_name)

            if os.path.isfile(emb_path) and os.path.isfile(vectorizer_path):
                with open(emb_path, "rb") as file:
                    self.p_embedding = pickle.load(file)
                with open(vectorizer_path, "rb") as file:
                    self.vectorizer = pickle.load(file)
                print(f"Passage embedding & Sparse Vectorizer Loaded from {retriever_path}")
        else:
            print(f'Initializing new sparse retriever')
            emb_path = os.path.join(output_path, pickle_name)
            vectorizer_path = os.path.join(output_path, vectorizer_name)

            # Transform by vectorizer
            if hasattr(import_module("sklearn.feature_extraction.text"), vectorizer_type):
                vectorizer_type = getattr(import_module("sklearn.feature_extraction.text"), vectorizer_type)
                self.vectorizer = vectorizer_type(tokenizer=tokenize_fn, **vectorizer_parameters)
                print(f'{self.vectorizer}')

            elif hasattr(import_module("retriever"), vectorizer_type):
                vectorizer_type = getattr(import_module("retriever"), vectorizer_type)
                self.vectorizer = vectorizer_type(tokenize_fn, vectorizer_parameters)
                print(f'{self.vectorizer}')
            else:
                raise Exception(f"Use correct tokenizer type : Current tokenizer : {vectorizer_type}")

            print("Build passage embedding")
            self.p_embedding = self.vectorizer.fit_transform(self.contexts)
            print(self.p_embedding.shape)
            with open(emb_path, "wb") as file:
                pickle.dump(self.p_embedding, file)
            with open(vectorizer_path, "wb") as file:
                pickle.dump(self.vectorizer, file)
            print(f"Saving Passage embedding & Sparse Vectorizer to {output_path}")


    def build_faiss(self) -> NoReturn:

        """
        Summary:
            속성으로 저장되어 있는 Passage Embedding을
            Faiss indexer에 fitting 시켜놓습니다.
            이렇게 저장된 indexer는 `get_relevant_doc`에서 유사도를 계산하는데 사용됩니다.

        Note:
            Faiss는 Build하는데 시간이 오래 걸리기 때문에,
            매번 새롭게 build하는 것은 비효율적입니다.
            그렇기 때문에 build된 index 파일을 저정하고 다음에 사용할 때 불러옵니다.
            다만 이 index 파일은 용량이 1.4Gb+ 이기 때문에 여러 num_clusters로 시험해보고
            제일 적절한 것을 제외하고 모두 삭제하는 것을 권장합니다.
        """
        num_clusters = self.num_clusters
        indexer_name = f"faiss_clusters{num_clusters}.index"
        indexer_path = os.path.join(self.data_path, indexer_name)
        if os.path.isfile(indexer_path):
            print("Load Saved Faiss Indexer.")
            self.indexer = faiss.read_index(indexer_path)

        else:
            p_emb = self.p_embedding.astype(np.float32).toarray()
            emb_dim = p_emb.shape[-1]

            num_clusters = num_clusters
            quantizer = faiss.IndexFlatL2(emb_dim)

            self.indexer = faiss.IndexIVFScalarQuantizer(
                quantizer, quantizer.d, num_clusters, faiss.METRIC_L2
            )
            self.indexer.train(p_emb)
            self.indexer.add(p_emb)
            faiss.write_index(self.indexer, indexer_path)
            print("Faiss Indexer Saved.")

    def retrieve(
        self, query_or_dataset: Union[str, Dataset], topk: Optional[int] = 1
    ) -> Union[Tuple[List, List], pd.DataFrame]:

        """
        Arguments:
            query_or_dataset (Union[str, Dataset]):
                str이나 Dataset으로 이루어진 Query를 받습니다.
                str 형태인 하나의 query만 받으면 `get_relevant_doc`을 통해 유사도를 구합니다.
                Dataset 형태는 query를 포함한 HF.Dataset을 받습니다.
                이 경우 `get_relevant_doc_bulk`를 통해 유사도를 구합니다.
            topk (Optional[int], optional): Defaults to 1.
                상위 몇 개의 passage를 사용할 것인지 지정합니다.

        Returns:
            1개의 Query를 받는 경우  -> Tuple(List, List)
            다수의 Query를 받는 경우 -> pd.DataFrame: [description]

        Note:
            다수의 Query를 받는 경우,
                Ground Truth가 있는 Query (train/valid) -> 기존 Ground Truth Passage를 같이 반환합니다.
                Ground Truth가 없는 Query (test) -> Retrieval한 Passage만 반환합니다.
        """

        assert self.p_embedding is not None, "get_sparse_embedding() 메소드를 먼저 수행해줘야합니다."

        if isinstance(query_or_dataset, str):
            doc_scores, doc_indices = self.get_relevant_doc(query_or_dataset, k=topk)
            print("[Search query]\n", query_or_dataset, "\n")

            for i in range(topk):
                print(f"Top-{i+1} passage with score {doc_scores[i]:4f}")
                print(self.contexts[doc_indices[i]])

            return (doc_scores, [self.contexts[doc_indices[i]] for i in range(topk)])

        elif isinstance(query_or_dataset, Dataset):

            # Retrieve한 Passage를 pd.DataFrame으로 반환합니다.
            total = []
            with timer("query exhaustive search"):
                doc_scores, doc_indices = self.get_relevant_doc_bulk(
                    query_or_dataset["question"], k=topk
                ) 
            for idx, example in enumerate(
                tqdm(query_or_dataset, desc="Sparse retrieval: ")
            ):
                tmp = {
                    # Query와 해당 id를 반환합니다.
                    "question": example["question"],
                    "id": example["id"],
                    # Retrieve한 Passage의 id, context를 반환합니다.
                    "context_id": doc_indices[idx],
                    "context": " ".join(
                        [self.contexts[pid] for pid in doc_indices[idx]]
                    ),
                }
                if "context" in example.keys() and "answers" in example.keys():
                    # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
                    tmp["original_context"] = example["context"]
                    tmp["answers"] = example["answers"]
                total.append(tmp)

            cqas = pd.DataFrame(total)
            return cqas

    def get_relevant_doc(self, query: str, k: Optional[int] = 1) -> Tuple[List, List]:

        """
        Arguments:
            query (str):
                하나의 Query를 받습니다.
            k (Optional[int]): 1
                상위 몇 개의 Passage를 반환할지 정합니다.
        Note:
            vocab 에 없는 이상한 단어로 query 하는 경우 assertion 발생 (예) 뙣뙇?
        """

        with timer("transform"):
            query_vec = self.vectorizer.transform([query])
        assert (
            np.sum(query_vec) != 0
        ), "오류가 발생했습니다. 이 오류는 보통 query에 vectorizer의 vocab에 없는 단어만 존재하는 경우 발생합니다."

        with timer("query ex search"):
            result = query_vec * self.p_embedding.T
        if not isinstance(result, np.ndarray):
            result = result.toarray()

        sorted_result = np.argsort(result.squeeze())[::-1]
        doc_score = result.squeeze()[sorted_result].tolist()[:k]
        doc_indices = sorted_result.tolist()[:k]
        return doc_score, doc_indices

    def get_relevant_doc_bulk(
        self, queries: List, k: Optional[int] = 1
    ) -> Tuple[List, List]:

        """
        Arguments:
            queries (List):
                하나의 Query를 받습니다.
            k (Optional[int]): 1
                상위 몇 개의 Passage를 반환할지 정합니다.
        Note:
            vocab 에 없는 이상한 단어로 query 하는 경우 assertion 발생 (예) 뙣뙇?
        """

        query_vec = self.vectorizer.transform(queries)
        assert (
            np.sum(query_vec) != 0
        ), "오류가 발생했습니다. 이 오류는 보통 query에 vectorizer의 vocab에 없는 단어만 존재하는 경우 발생합니다."

        result = query_vec * self.p_embedding.T
        if not isinstance(result, np.ndarray):
            result = result.toarray()
        doc_scores = []
        doc_indices = []
        for i in range(result.shape[0]):
            sorted_result = np.argsort(result[i, :])[::-1]
            doc_scores.append(result[i, :][sorted_result].tolist()[:k])
            doc_indices.append(sorted_result.tolist()[:k])
        return doc_scores, doc_indices

    def retrieve_faiss(
        self, query_or_dataset: Union[str, Dataset], topk: Optional[int] = 1
    ) -> Union[Tuple[List, List], pd.DataFrame]:

        """
        Arguments:
            query_or_dataset (Union[str, Dataset]):
                str이나 Dataset으로 이루어진 Query를 받습니다.
                str 형태인 하나의 query만 받으면 `get_relevant_doc`을 통해 유사도를 구합니다.
                Dataset 형태는 query를 포함한 HF.Dataset을 받습니다.
                이 경우 `get_relevant_doc_bulk`를 통해 유사도를 구합니다.
            topk (Optional[int], optional): Defaults to 1.
                상위 몇 개의 passage를 사용할 것인지 지정합니다.

        Returns:
            1개의 Query를 받는 경우  -> Tuple(List, List)
            다수의 Query를 받는 경우 -> pd.DataFrame: [description]

        Note:
            다수의 Query를 받는 경우,
                Ground Truth가 있는 Query (train/valid) -> 기존 Ground Truth Passage를 같이 반환합니다.
                Ground Truth가 없는 Query (test) -> Retrieval한 Passage만 반환합니다.
            retrieve와 같은 기능을 하지만 faiss.indexer를 사용합니다.
        """

        assert self.indexer is not None, "build_faiss()를 먼저 수행해주세요."

        if isinstance(query_or_dataset, str):
            doc_scores, doc_indices = self.get_relevant_doc_faiss(
                query_or_dataset, k=topk
            )
            print("[Search query]\n", query_or_dataset, "\n")

            for i in range(topk):
                print("Top-%d passage with score %.4f" % (i + 1, doc_scores[i]))
                print(self.contexts[doc_indices[i]])

            return (doc_scores, [self.contexts[doc_indices[i]] for i in range(topk)])

        elif isinstance(query_or_dataset, Dataset):

            # Retrieve한 Passage를 pd.DataFrame으로 반환합니다.
            queries = query_or_dataset["question"]
            total = []

            with timer("query faiss search"):
                doc_scores, doc_indices = self.get_relevant_doc_bulk_faiss(
                    queries, k=topk
                )
            for idx, example in enumerate(
                tqdm(query_or_dataset, desc="Sparse retrieval: ")
            ):
                tmp = {
                    # Query와 해당 id를 반환합니다.
                    "question": example["question"],
                    "id": example["id"],
                    # Retrieve한 Passage의 id, context를 반환합니다.
                    "context_id": doc_indices[idx],
                    "context": " ".join(
                        [self.contexts[pid] for pid in doc_indices[idx]]
                    ),
                }
                if "context" in example.keys() and "answers" in example.keys():
                    # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
                    tmp["original_context"] = example["context"]
                    tmp["answers"] = example["answers"]
                total.append(tmp)

            return pd.DataFrame(total)

    def get_relevant_doc_faiss(
        self, query: str, k: Optional[int] = 1
    ) -> Tuple[List, List]:

        """
        Arguments:
            query (str):
                하나의 Query를 받습니다.
            k (Optional[int]): 1
                상위 몇 개의 Passage를 반환할지 정합니다.
        Note:
            vocab 에 없는 이상한 단어로 query 하는 경우 assertion 발생 (예) 뙣뙇?
        """

        query_vec = self.vectorizer.transform([query])
        assert (
            np.sum(query_vec) != 0
        ), "오류가 발생했습니다. 이 오류는 보통 query에 vectorizer의 vocab에 없는 단어만 존재하는 경우 발생합니다."

        q_emb = query_vec.toarray().astype(np.float32)
        with timer("query faiss search"):
            D, I = self.indexer.search(q_emb, k)

        return D.tolist()[0], I.tolist()[0]

    def get_relevant_doc_bulk_faiss(
        self, queries: List, k: Optional[int] = 1
    ) -> Tuple[List, List]:

        """
        Arguments:
            queries (List):
                하나의 Query를 받습니다.
            k (Optional[int]): 1
                상위 몇 개의 Passage를 반환할지 정합니다.
        Note:
            vocab 에 없는 이상한 단어로 query 하는 경우 assertion 발생 (예) 뙣뙇?
        """

        query_vecs = self.vectorizer.transform(queries)
        assert (
            np.sum(query_vecs) != 0
        ), "오류가 발생했습니다. 이 오류는 보통 query에 vectorizer의 vocab에 없는 단어만 존재하는 경우 발생합니다."

        q_embs = query_vecs.toarray().astype(np.float32)
        D, I = self.indexer.search(q_embs, k)

        return D.tolist(), I.tolist()

In [ ]:
args.retriever_path

In [ ]:
retriever = SparseRetrieval(
    retrieval_path = args.retriever_path,
    retrieval_type=args.vectorizer_type,
    retrieval_parameters=args.vectorizer_parameters,
    tokenize_fn=tokenizer.tokenize if args.tokenizer_type == "AutoTokenizer" else tokenizer.morphs,
    output_path=output_path,
    data_path=args.data_path,
    context_path=args.context_path,
    num_clusters = args.num_clusters
)

In [ ]:
query = "대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?"

In [ ]:
with timer("single query by exhaustive search"):
    scores, indices = retriever.retrieve(query, topk=args.top_k)

In [ ]:
#
len(full_ds)

In [ ]:
query_or_dataset=full_ds
print(query_or_dataset['question'][0])
print(query_or_dataset['answers'][0])

In [ ]:
doc_scores, doc_indices = retriever.get_relevant_doc_bulk(query_or_dataset['question'][:3], k=20)

In [ ]:
print(len(doc_scores), len(doc_scores[0]))
print(len(doc_indices), len(doc_indices[0]))

In [ ]:
for idx, example in enumerate(tqdm(query_or_dataset, desc="Sparse retrieval: ")):
    pprint(example)
    print('====')
    print(f'"question": {example["question"]}')
    print(f'"id": {example["id"]}')
    print(f'"context_id": {doc_indices[idx]}')
    print(f'"answers": {example["answers"]}')
    break

In [ ]:
k = " ".join([retriever.contexts[pid] for pid in doc_indices[idx]])

In [ ]:
k